In [ ]:
# Importamos las librerías necesarias
import pandas as pd # Manejo de datos

In [ ]:
# Definimos las rutas de los archivos CSV
ruta_pobreza = '../../data/processed_data/personas_en_pobreza.csv'
ruta_abandonos_escolares = '../../data/processed_data/abandono_escolar_nivel_superior_2024.csv'
ruta_estudiantes_municipio = '../../data/processed_data/estudiantes_municipio_todos_los_estados.csv'
ruta_gasto_prom_entidad = '../../data/processed_data/gasto_promedio_trim_por_entidad_2024.csv'
ruta_ing_hogares = '../../data/processed_data/ing_hogares_por_entidad_2024.csv'
# definimos todos los dataframes
df_pobreza = pd.read_csv(ruta_pobreza)
df_abandonos_escolares = pd.read_csv(ruta_abandonos_escolares)
df_estudiantes_municipio = pd.read_csv(ruta_estudiantes_municipio)
df_gasto_prom_entidad = pd.read_csv(ruta_gasto_prom_entidad)
df_ing_hogares = pd.read_csv(ruta_ing_hogares)

In [ ]:
# Seccion comentada para corregir nombres de entidades
# # Diccionario de mapeo para corregir nombres de entidades
# correcciones_entidades = {
#     "Coahuila": "Coahuila de Zaragoza",
#     "Michoacán": "Michoacán de Ocampo",
#     "Estado de México": "México",
#     "Veracruz": "Veracruz de Ignacio de la Llave"
# }

# # Aplicar las correcciones en los DataFrames
# df_ing_hogares['NOM_ENT'] = df_ing_hogares['NOM_ENT'].replace(correcciones_entidades)
# df_gasto_prom_entidad['NOM_ENT'] = df_gasto_prom_entidad['NOM_ENT'].replace(correcciones_entidades)

In [ ]:
# Mostramos las primeras filas del DataFrame resultante
df_estudiantes_municipio.head(3)

,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,P_18A24
0,Aguascalientes,Aguascalientes,Total del Municipio,948990,120452
1,Aguascalientes,Asientos,Total del Municipio,51536,6581
2,Aguascalientes,Calvillo,Total del Municipio,58250,6935


## Primer MERGE entre estudiantes por municipio e ingreso de hogares

In [ ]:
# Primero unimos los datos de población y los ingresos
df_merge1 = df_estudiantes_municipio.merge(
    df_ing_hogares,
    how='left',  
    on = ['NOM_ENT']
)
df_merge1.head(3)
print(f'Dimensiones del DataFrame después del primer merge: {df_merge1.shape}, '
      f'Conteo de nulos: {df_merge1.isnull().sum().sum()}')

Dimensiones del DataFrame después del primer merge: (2469, 6), Conteo de nulos: 0


In [ ]:
# Calculamos la proporción de la población de 18 a 24 años estudianda y el ingreso del hogar ajustado
df_merge1['PROP_POBLACION'] = df_merge1['P18A24A'] / df_merge1['POBTOT']
df_merge1['INGRESO_HOGAR_TRI'] = df_merge1['INGRESO_TRI'] * df_merge1['PROP_POBLACION']
df_merge1.head(3)

,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,P_18A24,INGRESO_TRI,PROP_POBLACION,INGRESO_HOGAR_TRI
0,Aguascalientes,Aguascalientes,Total del Municipio,948990,120452,2.612018e+06,0.126927,331534.312190
1,Aguascalientes,Asientos,Total del Municipio,51536,6581,2.612018e+06,0.127697,333547.190078
2,Aguascalientes,Calvillo,Total del Municipio,58250,6935,2.612018e+06,0.119056,310975.832223


## Segundo MERGE con gasto promedio

In [ ]:
# Ahora unimos con el gasto promedio por entidad
df_merge2 = df_merge1.merge(df_gasto_prom_entidad,
    how='left',
    on = ['NOM_ENT']
)
df_merge2.head(3)
print(f"""Dimensiones del DataFrame después del segundo merge: {df_merge2.shape},
      'Conteo de nulos: {df_merge2.isnull().sum().sum()}""")

Dimensiones del DataFrame después del segundo merge: (2469, 9),
      f'Conteo de nulos: 0)


In [ ]:
# Calculamos el gasto municipal trimestral ajustado
df_merge2['GASTO_MUN_TRI'] = df_merge2['GASTO_PROM_TRI'] * df_merge2['PROP_POBLACION']
df_merge2.head(3)

,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,P_18A24,INGRESO_TRI,PROP_POBLACION,INGRESO_HOGAR_TRI,GASTO_PROM_TRI,GASTO_MUN_TRI
0,Aguascalientes,Aguascalientes,Total del Municipio,948990,120452,2.612018e+06,0.126927,331534.312190,182453.902356,23158.239230
1,Aguascalientes,Asientos,Total del Municipio,51536,6581,2.612018e+06,0.127697,333547.190078,182453.902356,23298.842196
2,Aguascalientes,Calvillo,Total del Municipio,58250,6935,2.612018e+06,0.119056,310975.832223,182453.902356,21722.194212


## Tercer MERGE con pobreza 

In [90]:
df_pobreza.head(3)

,NOM_ENT,NOM_MUN,POBR_CAR_PROM,VUL_CAR_PROM,VUL_ING_POR,CAR_SALUD_PROM,CAR_SEG_SOC_PROM
0,Aguascalientes,Aguascalientes,1.7831169668,1.5840867901,9.9786625324,2.2970483279,1.8780160657
1,Aguascalientes,Asientos,1.9178116878,1.6959376909,6.6190835444,2.6465038509,1.9489284058
2,Aguascalientes,Calvillo,1.8588923648,1.6259272309,3.7603158327,2.5111797362,1.8107633172


In [ ]:
# Finalmente unimos con los datos de pobreza
df_merge3 = df_merge2.merge(df_pobreza,
                            how = 'left',
                            on = ['NOM_ENT', 'NOM_MUN'])
print(f"""Dimensiones del DataFrame después del tercer merge: {df_merge3.shape},
        'Conteo de nulos: {df_merge3.isnull().sum()}""")

Dimensiones del DataFrame después del tercer merge: (2469, 15),
        'Conteo de nulos: NOM_ENT               0
NOM_MUN               0
NOM_LOC               0
POBTOT                0
P_18A24               0
INGRESO_TRI           0
PROP_POBLACION        0
INGRESO_HOGAR_TRI     0
GASTO_PROM_TRI        0
GASTO_MUN_TRI         0
POBR_CAR_PROM        29
VUL_CAR_PROM         29
VUL_ING_POR          29
CAR_SALUD_PROM       29
CAR_SEG_SOC_PROM     29
dtype: int64


In [91]:
df_merge3.head()

,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,P_18A24,INGRESO_TRI,PROP_POBLACION,INGRESO_HOGAR_TRI,GASTO_PROM_TRI,GASTO_MUN_TRI,POBR_CAR_PROM,VUL_CAR_PROM,VUL_ING_POR,CAR_SALUD_PROM,CAR_SEG_SOC_PROM
0,Aguascalientes,Aguascalientes,Total del Municipio,948990,120452,2.612018e+06,0.126927,331534.312190,182453.902356,23158.239230,1.7831169668,1.5840867901,9.9786625324,2.2970483279,1.8780160657
1,Aguascalientes,Asientos,Total del Municipio,51536,6581,2.612018e+06,0.127697,333547.190078,182453.902356,23298.842196,1.9178116878,1.6959376909,6.6190835444,2.6465038509,1.9489284058
2,Aguascalientes,Calvillo,Total del Municipio,58250,6935,2.612018e+06,0.119056,310975.832223,182453.902356,21722.194212,1.8588923648,1.6259272309,3.7603158327,2.5111797362,1.8107633172
3,Aguascalientes,Cosío,Total del Municipio,17000,2228,2.612018e+06,0.131059,342327.956925,182453.902356,23912.193791,1.8019980186,1.6389019914,9.9296487621,2.6258059325,1.9443049472
4,Aguascalientes,Jesús María,Total del Municipio,129929,16640,2.612018e+06,0.128070,334520.955862,182453.902356,23366.861403,2.0239337363,1.6226115722,7.7033078899,2.5178721773,2.0297539465


In [ ]:
# Manejamos los valores nulos en las columnas de pobreza
cols_to_fix = ['POBR_CAR_PROM', 'VUL_CAR_PROM', 'VUL_ING_POR', 'CAR_SALUD_PROM', 'CAR_SEG_SOC_PROM']
for col in cols_to_fix:
	df_merge3[col] = pd.to_numeric(df_merge3[col], errors='coerce')
	df_merge3[col] = df_merge3[col].fillna(df_merge3[col].mean())
df_merge3.head(3)

,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,P_18A24,INGRESO_TRI,PROP_POBLACION,INGRESO_HOGAR_TRI,GASTO_PROM_TRI,GASTO_MUN_TRI,POBR_CAR_PROM,VUL_CAR_PROM,VUL_ING_POR,CAR_SALUD_PROM,CAR_SEG_SOC_PROM
0,Aguascalientes,Aguascalientes,Total del Municipio,948990,120452,2.612018e+06,0.126927,331534.312190,182453.902356,23158.239230,1.783117,1.584087,9.978663,2.297048,1.878016
1,Aguascalientes,Asientos,Total del Municipio,51536,6581,2.612018e+06,0.127697,333547.190078,182453.902356,23298.842196,1.917812,1.695938,6.619084,2.646504,1.948928
2,Aguascalientes,Calvillo,Total del Municipio,58250,6935,2.612018e+06,0.119056,310975.832223,182453.902356,21722.194212,1.858892,1.625927,3.760316,2.511180,1.810763


In [93]:
df_merge3.isnull().sum()

NOM_ENT              0
NOM_MUN              0
NOM_LOC              0
POBTOT               0
P_18A24              0
INGRESO_TRI          0
PROP_POBLACION       0
INGRESO_HOGAR_TRI    0
GASTO_PROM_TRI       0
GASTO_MUN_TRI        0
POBR_CAR_PROM        0
VUL_CAR_PROM         0
VUL_ING_POR          0
CAR_SALUD_PROM       0
CAR_SEG_SOC_PROM     0
dtype: int64

## Cuarto MERGE con abandano escolar 

In [94]:
df_abandonos_escolares.head()

,NOM_ENT,Nivel,TASA_ABANDONO
0,Aguascalientes,Superior,8.8
1,Baja California,Superior,5.1
2,Baja California Sur,Superior,13.6
3,Campeche,Superior,12.2
4,Coahuila de Zaragoza,Superior,6.4


In [ ]:
# Finalmente unimos con los datos de abandono escolar
df_merge4 = df_merge3.merge(df_abandonos_escolares,
                            how = 'left',
                            on = ['NOM_ENT'])
print(f"""Dimensiones del DataFrame después del cuarto merge: {df_merge4.shape},
        'Conteo de nulos: {df_merge4.isnull().sum()}""")

Dimensiones del DataFrame después del cuarto merge: (2469, 17),
        'Conteo de nulos: NOM_ENT              0
NOM_MUN              0
NOM_LOC              0
POBTOT               0
P_18A24              0
INGRESO_TRI          0
PROP_POBLACION       0
INGRESO_HOGAR_TRI    0
GASTO_PROM_TRI       0
GASTO_MUN_TRI        0
POBR_CAR_PROM        0
VUL_CAR_PROM         0
VUL_ING_POR          0
CAR_SALUD_PROM       0
CAR_SEG_SOC_PROM     0
Nivel                0
TASA_ABANDONO        0
dtype: int64


In [96]:
df_merge4.head(2)

,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,P_18A24,INGRESO_TRI,PROP_POBLACION,INGRESO_HOGAR_TRI,GASTO_PROM_TRI,GASTO_MUN_TRI,POBR_CAR_PROM,VUL_CAR_PROM,VUL_ING_POR,CAR_SALUD_PROM,CAR_SEG_SOC_PROM,Nivel,TASA_ABANDONO
0,Aguascalientes,Aguascalientes,Total del Municipio,948990,120452,2.612018e+06,0.126927,331534.312190,182453.902356,23158.239230,1.783117,1.584087,9.978663,2.297048,1.878016,Superior,8.8
1,Aguascalientes,Asientos,Total del Municipio,51536,6581,2.612018e+06,0.127697,333547.190078,182453.902356,23298.842196,1.917812,1.695938,6.619084,2.646504,1.948928,Superior,8.8


In [ ]:
# Eliminamos las columnas que ya no son necesarias
df_merge4.drop(columns = ['POBTOT', 'P18A24A', 'INGRESO_TRI', 'PROP_POBLACION', 'GASTO_PROM_TRI', 'NOM_LOC', 'Nivel'], inplace=True)
df_merge4.head(3)

,NOM_ENT,NOM_MUN,INGRESO_HOGAR_TRI,GASTO_MUN_TRI,POBR_CAR_PROM,VUL_CAR_PROM,VUL_ING_POR,CAR_SALUD_PROM,CAR_SEG_SOC_PROM,TASA_ABANDONO
0,Aguascalientes,Aguascalientes,331534.312190,23158.239230,1.783117,1.584087,9.978663,2.297048,1.878016,8.8
1,Aguascalientes,Asientos,333547.190078,23298.842196,1.917812,1.695938,6.619084,2.646504,1.948928,8.8
2,Aguascalientes,Calvillo,310975.832223,21722.194212,1.858892,1.625927,3.760316,2.511180,1.810763,8.8


In [98]:
df_merge4.to_csv('..\\data\\data_final_proyecto_final.csv', index=False)